In [1]:
import sys
sys.path.append("../src/")
import data_loader
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import nltk
#nltk.download('wordnet')
#nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import pandas as pd
import re
from collections import Counter
import joblib

In [2]:
train_data,test_data = data_loader.load_data('../data/raw')
train_data.head(10)

,review,sentiment
0,I caught this little gem totally by accident b...,positive
1,I can't believe that I let myself into this mo...,negative
2,*spoiler alert!* it just gets to me the nerve ...,negative
3,If there's one thing I've learnt from watching...,negative
4,"I remember when this was in theaters, reviews ...",negative
5,Opera (the U.S. title is terror at the opera) ...,positive
6,Heard about this film a long while ago and fin...,positive
7,"John Holmes is so famous, he's infamous (as th...",positive
8,What is worth mentioning that is omitted in th...,positive
9,Darling Lili is fantastic! Its by far one my f...,positive


In [3]:
train_data['review'][5]

"Opera (the U.S. title is terror at the opera) is somewhat of a letdown after some of Dario's other movies like Phenomena, Tenebre, and Suspiria. (i still can't find Inferno anywhere.) it's one of those movies that has a great first half but midway through it's like someone started slowly letting the air out of the screenplay and logic.<br /><br />the basic plot involves a beautiful opera singer who is being stalked by a deranged obsessed fan. this killer begins killing people close to her in a most unique fashion. he binds and gags her and tape tiny sharp pins under her eyelids so if she tries to close her eyes she'll gouge out her eyes. this forces her to watch while the killer murders her acquaintances in typically brutal and gory Argento fashion.<br /><br />unfortunately, about midway through the film becomes sluggish and illogical. (this is especially directed towards the killer's motivations. i still haven't completely figured out why he's such a nut.) the ending especially come 

In [4]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     40000 non-null  object
 1   sentiment  40000 non-null  object
dtypes: object(2)
memory usage: 625.1+ KB


In [5]:
train_data.isnull().sum()

review       0
sentiment    0
dtype: int64

In [6]:
train_data['sentiment'].value_counts()

positive    20000
negative    20000
Name: sentiment, dtype: int64

In [7]:
train_data['review_length'] = train_data['review'].str.len()
train_data.head(10)

,review,sentiment,review_length
0,I caught this little gem totally by accident b...,positive,892
1,I can't believe that I let myself into this mo...,negative,937
2,*spoiler alert!* it just gets to me the nerve ...,negative,468
3,If there's one thing I've learnt from watching...,negative,1960
4,"I remember when this was in theaters, reviews ...",negative,1015
5,Opera (the U.S. title is terror at the opera) ...,positive,1241
6,Heard about this film a long while ago and fin...,positive,1127
7,"John Holmes is so famous, he's infamous (as th...",positive,3182
8,What is worth mentioning that is omitted in th...,positive,2270
9,Darling Lili is fantastic! Its by far one my f...,positive,1587


In [8]:
def word_count(review):
    review_list = review.split()
    return len(review_list)

train_data['word_count'] = train_data['review'].apply(word_count)
train_data.head(10)

,review,sentiment,review_length,word_count
0,I caught this little gem totally by accident b...,positive,892,161
1,I can't believe that I let myself into this mo...,negative,937,170
2,*spoiler alert!* it just gets to me the nerve ...,negative,468,69
3,If there's one thing I've learnt from watching...,negative,1960,314
4,"I remember when this was in theaters, reviews ...",negative,1015,183
5,Opera (the U.S. title is terror at the opera) ...,positive,1241,211
6,Heard about this film a long while ago and fin...,positive,1127,210
7,"John Holmes is so famous, he's infamous (as th...",positive,3182,570
8,What is worth mentioning that is omitted in th...,positive,2270,410
9,Darling Lili is fantastic! Its by far one my f...,positive,1587,267


In [9]:
train_data['mean_word_length'] = train_data['review'].map(lambda rev: np.mean([len(word) for word in rev.split()]))
train_data.head(10)

,review,sentiment,review_length,word_count,mean_word_length
0,I caught this little gem totally by accident b...,positive,892,161,4.546584
1,I can't believe that I let myself into this mo...,negative,937,170,4.517647
2,*spoiler alert!* it just gets to me the nerve ...,negative,468,69,5.797101
3,If there's one thing I've learnt from watching...,negative,1960,314,5.245223
4,"I remember when this was in theaters, reviews ...",negative,1015,183,4.551913
5,Opera (the U.S. title is terror at the opera) ...,positive,1241,211,4.886256
6,Heard about this film a long while ago and fin...,positive,1127,210,4.371429
7,"John Holmes is so famous, he's infamous (as th...",positive,3182,570,4.584211
8,What is worth mentioning that is omitted in th...,positive,2270,410,4.539024
9,Darling Lili is fantastic! Its by far one my f...,positive,1587,267,4.947566


In [10]:
train_data['mean_sentence_length'] = train_data['review'].map(lambda rev: np.mean([len(sent) for sent in nltk.tokenize.sent_tokenize(rev)]))
train_data.head(10)

,review,sentiment,review_length,word_count,mean_word_length,mean_sentence_length
0,I caught this little gem totally by accident b...,positive,892,161,4.546584,67.692308
1,I can't believe that I let myself into this mo...,negative,937,170,4.517647,186.600000
2,*spoiler alert!* it just gets to me the nerve ...,negative,468,69,5.797101,66.142857
3,If there's one thing I've learnt from watching...,negative,1960,314,5.245223,653.000000
4,"I remember when this was in theaters, reviews ...",negative,1015,183,4.551913,100.600000
5,Opera (the U.S. title is terror at the opera) ...,positive,1241,211,4.886256,123.200000
6,Heard about this film a long while ago and fin...,positive,1127,210,4.371429,124.333333
7,"John Holmes is so famous, he's infamous (as th...",positive,3182,570,4.584211,101.677419
8,What is worth mentioning that is omitted in th...,positive,2270,410,4.539024,453.200000
9,Darling Lili is fantastic! Its by far one my f...,positive,1587,267,4.947566,87.333333


In [11]:
STOPWORDS = set(stopwords.words('english'))
cnt = Counter()
stemmer = PorterStemmer()
lemma = WordNetLemmatizer()

def clean_review_with_lemmatizer(review_text):
    review_text = str(review_text).lower()
    review_text = re.sub('[^a-zA-Z]',' ',review_text)
    review_text = word_tokenize(review_text)
    review_text = [word for word in review_text if word not in STOPWORDS]
    review_text = [lemma.lemmatize(word=w,pos='v') for w in review_text]
    review_text = [word for word in review_text if len(word) > 2]
    review_text = ' '.join(review_text)
    return review_text

def clean_review_with_stemmer(review_text):
    review_text = str(review_text).lower()
    review_text = re.sub('[^a-zA-Z]',' ',review_text)
    review_text = word_tokenize(review_text)
    review_text = [word for word in review_text if word not in STOPWORDS]
    review_text = [stemmer.stem(w) for w in review_text]
    review_text = [word for word in review_text if len(word) > 2]
    review_text = ' '.join(review_text)
    return review_text

train_data['Clean_reviews_with_lemm'] = train_data['review'].apply(clean_review_with_lemmatizer)
train_data['Clean_reviews_with_stem'] = train_data['review'].apply(clean_review_with_stemmer)
train_data.head()

,review,sentiment,review_length,word_count,mean_word_length,mean_sentence_length,Clean_reviews_with_lemm,Clean_reviews_with_stem
0,I caught this little gem totally by accident b...,positive,892,161,4.546584,67.692308,catch little gem totally accident back revival...,caught littl gem total accid back reviv theatr...
1,I can't believe that I let myself into this mo...,negative,937,170,4.517647,186.600000,believe let movie accomplish favor friends ask...,believ let movi accomplish favor friend ask ea...
2,*spoiler alert!* it just gets to me the nerve ...,negative,468,69,5.797101,66.142857,spoiler alert get nerve people remake use term...,spoiler alert get nerv peopl remak use term lo...
3,If there's one thing I've learnt from watching...,negative,1960,314,5.245223,653.000000,one thing learn watch george romero creepshow ...,one thing learnt watch georg romero creepshow ...
4,"I remember when this was in theaters, reviews ...",negative,1015,183,4.551913,100.600000,remember theaters review say horrible well thi...,rememb theater review said horribl well think ...


In [12]:
from sklearn.model_selection import train_test_split

X_train_lemm, X_test_lemm, y_train, y_test = train_test_split(train_data['Clean_reviews_with_lemm'], train_data['sentiment'], test_size=0.2, random_state=42)
X_train_stem, X_test_stem, y_train, y_test = train_test_split(train_data['Clean_reviews_with_stem'], train_data['sentiment'], test_size=0.2, random_state=42)

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer_lemm = CountVectorizer(ngram_range=(1,2))
vectorized_lemm_reviews_train = vectorizer_lemm.fit_transform(X_train_lemm);
vectorized_lemm_reviews_test = vectorizer_lemm.transform(X_test_lemm);

vectorizer_stem = CountVectorizer(ngram_range=(1,2))
vectorized_stem_reviews_train = vectorizer_stem.fit_transform(X_train_stem)
vectorized_stem_reviews_test = vectorizer_stem.transform(X_test_stem)

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer_lemm = TfidfVectorizer(ngram_range=(1, 2))
X_train_lemm_tfidf = tfidf_vectorizer_lemm.fit_transform(X_train_lemm)
X_test_lemm_tfidf = tfidf_vectorizer_lemm.transform(X_test_lemm)

tfidf_vectorizer_stem = TfidfVectorizer(ngram_range=(1, 2))
X_train_stem_tfidf = tfidf_vectorizer_stem.fit_transform(X_train_stem)
X_test_stem_tfidf = tfidf_vectorizer_stem.transform(X_test_stem)

In [15]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

NB_model_lemm = MultinomialNB()
NB_model_lemm.fit(vectorized_lemm_reviews_train, y_train)

y_pred_lemm = NB_model_lemm.predict(vectorized_lemm_reviews_test)

nb_accuracy_lemm = accuracy_score(y_test, y_pred_lemm)
print("Accuracy with lemmatization:", nb_accuracy_lemm)

NB_model_stem = MultinomialNB()
NB_model_stem.fit(vectorized_stem_reviews_train, y_train)

y_pred_stem = NB_model_stem.predict(vectorized_stem_reviews_test)

nb_accuracy_stem = accuracy_score(y_test, y_pred_stem)
print("Accuracy with stemming:", nb_accuracy_stem)

Accuracy with lemmatization: 0.88125
Accuracy with stemming: 0.877625


In [16]:
from sklearn.svm import SVC

svm_model_lemm = SVC(C = 1.0,kernel='linear')
svm_model_lemm.fit(vectorized_lemm_reviews_train, y_train)

svm_pred_lemm = svm_model_lemm.predict(vectorized_lemm_reviews_test)

svm_accuracy_lemm = accuracy_score(y_test, svm_pred_lemm)
print("Accuracy with lemmatization:", svm_accuracy_lemm)

svm_model_stem = SVC(kernel='linear')
svm_model_stem.fit(vectorized_stem_reviews_train, y_train)

svm_pred_stem = svm_model_stem.predict(vectorized_stem_reviews_test)

svm_accuracy_stem = accuracy_score(y_test, svm_pred_stem)
print("Accuracy with stemming:", svm_accuracy_stem)

Accuracy with lemmatization: 0.891
Accuracy with stemming: 0.89


In [17]:
from sklearn.linear_model import LogisticRegression

LR_model_lemm = LogisticRegression(max_iter=500)
LR_model_lemm.fit(vectorized_lemm_reviews_train, y_train)

LR_pred_lemm = LR_model_lemm.predict(vectorized_lemm_reviews_test)
LR_accuracy_lemm = accuracy_score(y_test, LR_pred_lemm)
print("Accuracy with lemmatization:", LR_accuracy_lemm)

LR_model_stem = LogisticRegression(max_iter=500)
LR_model_stem.fit(vectorized_stem_reviews_train, y_train)

LR_pred_stem = LR_model_stem.predict(vectorized_stem_reviews_test)
LR_accuracy_stem = accuracy_score(y_test, LR_pred_stem)
print("Accuracy with stemming:", LR_accuracy_stem)

Accuracy with lemmatization: 0.895
Accuracy with stemming: 0.893


In [18]:
NB_model_lemm = MultinomialNB()
NB_model_lemm.fit(X_train_lemm_tfidf, y_train)

y_pred_lemm = NB_model_lemm.predict(X_test_lemm_tfidf)

nb_accuracy_lemm1 = accuracy_score(y_test, y_pred_lemm)
print("Accuracy with lemmatization:", nb_accuracy_lemm1)

NB_model_stem = MultinomialNB()
NB_model_stem.fit(X_train_stem_tfidf, y_train)

y_pred_stem = NB_model_stem.predict(X_test_stem_tfidf)

nb_accuracy_stem1 = accuracy_score(y_test, y_pred_stem)
print("Accuracy with stemming:", nb_accuracy_stem1)

Accuracy with lemmatization: 0.8845
Accuracy with stemming: 0.883625


In [19]:
svm_model_lemm = SVC(C=1.0, kernel='linear')
svm_model_lemm.fit(X_train_lemm_tfidf, y_train)

svm_pred_lemm = svm_model_lemm.predict(X_test_lemm_tfidf)

svm_accuracy_lemm1 = accuracy_score(y_test, svm_pred_lemm)
print("Accuracy with lemmatization:", svm_accuracy_lemm1)

svm_model_stem = SVC(C=1.0,kernel='linear')
svm_model_stem.fit(X_train_stem_tfidf, y_train)

svm_pred_stem = svm_model_stem.predict(X_test_stem_tfidf)

svm_accuracy_stem1 = accuracy_score(y_test, svm_pred_stem)
print("Accuracy with stemming:", svm_accuracy_stem1)

Accuracy with lemmatization: 0.90425
Accuracy with stemming: 0.903875


In [20]:
LR_model_lemm = LogisticRegression(max_iter=500)
LR_model_lemm.fit(X_train_lemm_tfidf, y_train)

LR_pred_lemm = LR_model_lemm.predict(X_test_lemm_tfidf)

LR_accuracy_lemm1 = accuracy_score(y_test, LR_pred_lemm)
print("Accuracy with lemmatization:", LR_accuracy_lemm1)

LR_model_stem = LogisticRegression(max_iter=500)
LR_model_stem.fit(X_train_stem_tfidf, y_train)

LR_pred_stem = LR_model_stem.predict(X_test_stem_tfidf)

LR_accuracy_stem1 = accuracy_score(y_test, LR_pred_stem)
print("Accuracy with stemming:", LR_accuracy_stem1)

Accuracy with lemmatization: 0.887125
Accuracy with stemming: 0.886875


In [21]:
print("##### Results with Count vectorizer ######")

df = pd.DataFrame(index=['Lemmatization','Stemming'], columns=['Naive Bayes','SVM','Logistic Regression'])
df.loc['Lemmatization', 'Naive Bayes'] = nb_accuracy_lemm * 100
df.loc['Lemmatization', 'SVM'] = svm_accuracy_lemm * 100
df.loc['Lemmatization', 'Logistic Regression'] = LR_accuracy_lemm * 100

df.loc['Stemming', 'Naive Bayes'] = nb_accuracy_stem * 100
df.loc['Stemming', 'SVM'] = svm_accuracy_stem * 100
df.loc['Stemming', 'Logistic Regression'] = LR_accuracy_stem * 100

print(df)

print("##### Results with TF-IDF vectorizer ######")

df1 = pd.DataFrame(index=['Lemmatization','Stemming'], columns=['Naive Bayes','SVM','Logistic Regression'])
df1.loc['Lemmatization', 'Naive Bayes'] = nb_accuracy_lemm1 * 100
df1.loc['Lemmatization', 'SVM'] = svm_accuracy_lemm1 * 100
df1.loc['Lemmatization', 'Logistic Regression'] = LR_accuracy_lemm1 * 100

df1.loc['Stemming', 'Naive Bayes'] = nb_accuracy_stem1 * 100
df1.loc['Stemming', 'SVM'] = svm_accuracy_stem1 * 100
df1.loc['Stemming', 'Logistic Regression'] = LR_accuracy_stem1 * 100

print(df1)

##### Results with Count vectorizer ######
              Naive Bayes   SVM Logistic Regression
Lemmatization      88.125  89.1                89.5
Stemming          87.7625  89.0                89.3
##### Results with TF-IDF vectorizer ######
              Naive Bayes      SVM Logistic Regression
Lemmatization       88.45   90.425             88.7125
Stemming          88.3625  90.3875             88.6875


In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf'],'gamma': [0.1, 0.01, 0.001, 0.0001]}
svm_model_lemm = SVC()
grid_search = GridSearchCV(svm_model_lemm, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train_lemm_tfidf, y_train)
print("Best parameters:", grid_search.best_params_)
print("Best accuracy:", grid_search.best_score_)
svm_pred_lemm_best = grid_search.best_estimator_.predict(X_test_lemm_tfidf)
svm_accuracy_lemm_best = accuracy_score(y_test, svm_pred_lemm_best)
print("Best accuracy with lemmatization:", svm_accuracy_lemm_best)